In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [3]:
dataset = 'Neuron_scNT'
root = '/scratch/blaauw_root/blaauw1/gyichen'
adata = anndata.read_h5ad(f'{root}/data/{dataset}_pp.h5ad')
adata.obs['clusters'] = adata.obs['time'].astype(str)

In [ ]:
vv.preprocess(adata, n_gene=2000, min_shared_counts=20, keep_raw=True)

In [4]:
model_path_base = f'{root}/checkpoints/{dataset}'
figure_path_base = f'{root}/figures/{dataset}'
data_path = f'{root}/data/velovae/discrete/{dataset}'
gene_plot = ['Fosb','Rfx3','Cebpg','Homer1']

# Informative Time Prior

In [5]:
adata.obs['tprior'] = adata.obs['time'].to_numpy().astype(float)

# Discrete VeloVAE

In [6]:
figure_path = f'{figure_path_base}/DVAE'
model_path = f'{model_path_base}/DVAE'

torch.manual_seed(2022)
np.random.seed(2022)
dvae = vv.VAE(adata, 
              tmax=20, 
              dim_z=5, 
              device='cuda:0',
              discrete=True,
              init_method='tprior',
              init_key='tprior',
              tprior='tprior',
              init_ton_zero=False)

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dvae', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Detected 34 velocity genes.
Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

Initialization using prior time.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Initial induction: 431, repression: 569/1000
Using informative time prior.
Learning Rate based on Data Sparsity: 0.0001
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********                      Stage  2                       *********
*********             Velocity Refinement Round 1             *********


  0%|          | 0/3066 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.053
Average Set Size: 72
*********     Round 1: Early Stop Triggered at epoch 1036.    *********
*********             Velocity Refinement Round 2             *********
*********     Round 2: Early Stop Triggered at epoch 1059.    *********
Change in x0: 0.7268
*********             Velocity Refinement Round 3             *********
*********     Round 3: Early Stop Triggered at epoch 1084.    *********
Change in x0: 0.5854
*********             Velocity Refinement Round 4             *********
*********     Round 4: Early Stop Triggered at epoch 1092.    *********
Change in x0: 0.5397
*********             Velocity Refinement Round 5             *********
Change in x0: 0.4824
*********             Velocity Refinement Round 6             *********
*********     Round 6: Early Stop Triggered at epoch 1606.    *********
Change in x0: 0.7995
*********             Velocity Refinement Round 7             *********
Stage 2: Early Stop Triggered at round 6.
******

# Discrete Full VB

In [8]:
figure_path = f'{figure_path_base}/DFullVB'
model_path = f'{model_path_base}/DFullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.VAE(adata, 
                 tmax=20, 
                 dim_z=5, 
                 device='cuda:0',
                 discrete=True,
                 full_vb=True,
                 init_method='tprior',
                 init_key='tprior',
                 tprior='tprior',
                 init_ton_zero=False)

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'dfullvb', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Detected 34 velocity genes.
Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

Initialization using prior time.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

Initial induction: 431, repression: 569/1000
Using informative time prior.
Learning Rate based on Data Sparsity: 0.0001
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********                      Stage  2                       *********
*********             Velocity Refinement Round 1             *********


  0%|          | 0/3066 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.080
Average Set Size: 85
*********     Round 1: Early Stop Triggered at epoch 1169.    *********
*********             Velocity Refinement Round 2             *********
*********     Round 2: Early Stop Triggered at epoch 1200.    *********
Change in x0: 0.5112
*********             Velocity Refinement Round 3             *********
*********     Round 3: Early Stop Triggered at epoch 1217.    *********
Change in x0: 0.3856
*********             Velocity Refinement Round 4             *********
*********     Round 4: Early Stop Triggered at epoch 1236.    *********
Change in x0: 0.3215
*********             Velocity Refinement Round 5             *********
*********     Round 5: Early Stop Triggered at epoch 1257.    *********
Change in x0: 0.2743
*********             Velocity Refinement Round 6             *********
*********     Round 6: Early Stop Triggered at epoch 1276.    *********
Change in x0: 0.2384
*********             Velocity Refinement Round 

# Evaluation

In [9]:
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE', 'Discrete FullVB'],
                 ['dvae', 'dfullvb'],
                 compute_metrics=False,
                 raw_count=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 figure_path=figure_path_base)

---   Plotting  Results   ---
computing velocity graph (using 1/32 cores)


  0%|          | 0/3066 [00:00<?, ?cells/s]

    finished (0:00:07) --> added 
    'dvae_velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:00) --> added
    'dvae_velocity_umap', embedded velocity vectors (adata.obsm)
saving figure to file /scratch/blaauw_root/blaauw1/gyichen/figures/Neuron_scNT/eval_dvae_stream.png
computing velocity graph (using 1/32 cores)


  0%|          | 0/3066 [00:00<?, ?cells/s]

    finished (0:00:07) --> added 
    'dfullvb_velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:00) --> added
    'dfullvb_velocity_umap', embedded velocity vectors (adata.obsm)
saving figure to file /scratch/blaauw_root/blaauw1/gyichen/figures/Neuron_scNT/eval_dfullvb_stream.png


(None, None)